In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [3]:
# Load the data
cost_matrix = pd.read_csv('/Users/mahinbindra/Downloads/cost_matrix.csv').to_numpy()
distributions = pd.read_csv('/Users/mahinbindra/Downloads/distributions.csv')

In [4]:
# Set up the model
model = gp.Model("Baffin_Bay_Retail_Network")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [5]:
# Number of retailers
n = len(distributions)

In [6]:
# Constants
selling_price = 49.99
ordering_cost = 24.44

In [7]:
# Decision variables
y = model.addVars(n, vtype=GRB.CONTINUOUS, name="y")
x = model.addVars(n, n, vtype=GRB.CONTINUOUS, name="x")

In [8]:
# Objective function
revenue = gp.quicksum(selling_price * distributions['Mean'][i] - ordering_cost * y[i] for i in range(n))
transshipment_cost = gp.quicksum(cost_matrix[i, j] * x[i, j] for i in range(n) for j in range(n) if i != j)
model.setObjective(revenue - transshipment_cost, GRB.MAXIMIZE)

In [9]:
# Constraints
# Each retailer's order should at least meet the expected demand minus the amount it transships to others
for i in range(n):
    model.addConstr(y[i] + gp.quicksum(x[j, i] for j in range(n) if j != i) >= distributions['Mean'][i])

# Transshipment from a retailer cannot exceed its excess over its own demand
for i in range(n):
    for j in range(n):
        if i != j:
            model.addConstr(x[i, j] <= y[i] - distributions['Mean'][i])

In [10]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 225 rows, 240 columns and 645 nonzeros
Model fingerprint: 0x69f646eb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 1e+02]
Presolve removed 0 rows and 15 columns
Presolve time: 0.01s
Presolved: 225 rows, 225 columns, 645 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5267443e+04   0.000000e+00   0.000000e+00      0s
       0    3.5267443e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.526744315e+04


In [11]:
# Output results
if model.status == GRB.OPTIMAL:
    print("Optimal value (profit): $", model.ObjVal)
    print("Order quantities (y_i):")
    for i in range(n):
        print(f"Retailer {i+1}: {y[i].X:.2f} units")

Optimal value (profit): $ 35267.443152940985
Order quantities (y_i):
Retailer 1: 89.89 units
Retailer 2: 51.82 units
Retailer 3: 99.10 units
Retailer 4: 66.75 units
Retailer 5: 53.45 units
Retailer 6: 61.77 units
Retailer 7: 114.65 units
Retailer 8: 110.04 units
Retailer 9: 99.58 units
Retailer 10: 98.32 units
Retailer 11: 82.44 units
Retailer 12: 95.27 units
Retailer 13: 120.12 units
Retailer 14: 121.85 units
Retailer 15: 115.28 units
